# 3D Plot of Autocorrelation

Before viewing centroids, I'd like to view them as 3D contour plots

In [ ]:
# Import Modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.fftpack import fft2,ifft2,fftshift
from speckle_fns import fits_import, circ_filter1
from speckle_fns import deconv0,deconv1, postprocess
from speckle_fns import filter_lpf, filter_hpf, filter_interference
import cv2

%matplotlib inline

In [ ]:
# Generating Autocorrelogram

# Filenames
filePathDouble = "/home/niels/Desktop/KP330_PSD.fits"
filePathSingle = "/home/niels/Desktop/KP331_PSD.fits"
# Import FITS file data
fitsDoubleStar = fits_import(filePathDouble)
fitsSingleStar = fits_import(filePathSingle)
# Convert FITS Data to Floats
psdDoubleStar = fitsDoubleStar.astype(float)
psdSingleStar = fitsSingleStar.astype(float)

# Perform Deconvolution
constant = 1E-15
imgF = fftshift(deconv1(psdDoubleStar, psdSingleStar, constant))

# Filter PSD:
imgF_filtered = filter_interference(imgF)
imgF_filtered = filter_lpf(imgF_filtered, 5)
imgF_filtered = filter_hpf(imgF_filtered, 6, 0.8)

# Postprocess
img_autocorr = postprocess(imgF_filtered)

# Plot Image
plt.figure(figsize = (6,6))
plt.imshow(img_autocorr)
plt.title("Autocorr of Filtered Image")
plt.colorbar()

# Create variables for surface plot
x = np.arange(512)
y = np.arange(512)
x,y = np.meshgrid(x,y)
z = img_autocorr

# Plot surface plot 
fig = plt.figure(figsize=(16,16))
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x,y,z,rstride=5,cstride=5, linewidth=0)

In [ ]:
# Using thresholding to try to find the peaks
# Start with the max value of the image (so I'd remove everything), and work my way down
#  until I have 3 well defined regions

max_pixel = np.max(z)
print(max_pixel)

threshold = 100

threshold_img = np.zeros(np.shape(z))

for row in range(np.shape(z)[0]):
    for column in range(np.shape(z)[0]):
        threshold_img[row,column] = (z[row,column]>threshold)

fig = plt.figure(figsize=(8,8))
plt.imshow(threshold_img)



In [ ]:
# Can I use HPF to try to find the edges of these circles?

edges_threshold=fftshift(fft2(threshold_img))

hpf_img = np.zeros(np.shape(edges_threshold))
hpf_img.fill(1)
hpf_img = filter_hpf(hpf_img,50,1)

edges_threshold = np.multiply(hpf_img,edges_threshold)

edges_threshold = (ifft2(edges_threshold))

plt.figure(figsize=(8,8))

plt.imshow(np.log10(np.abs(edges_threshold)))

# Doesn't look like this is too viable

In [ ]:
# Attempting to use openCV to find the 3 countours of thresholded 
threshold_img_scale = (np.multiply(255,threshold_img.astype(np.uint8)))

# Find Contours
im2,contours,hierarchy = cv2.findContours(threshold_img_scale,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# Display Contours
plt.figure(figsize=(20,7))
for i in np.arange(3):
    plt.subplot(1,3,i+1)
    
    # Draw contours on image
    mask = np.zeros((512,512))
    cv2.drawContours(mask,contours,i,(255,255,255),-1)
    plt.imshow(mask)

print("Number of Contours: ", len(contours))


In [ ]:
np.shape(mask)
print(mask[90,90])